In [25]:
import pandas as pd
import re
import os
os.chdir('C:/Users/admin/OneDrive/Documents/TrabajoTesis')

In [26]:
df = pd.read_excel("data/DatosPuertos.xlsx")

In [27]:
import pandas as pd

# Assuming your dataframe is called df
columns_to_keep = ['portNumber', 'portName', 'latitude', 'longitude', 'harborSize']

# Keep only these columns
df = df[columns_to_keep]

# Check the result
print(df.head())


   portNumber             portName    latitude   longitude harborSize
0       14660          Antofagasta  23°39'00"S  70°25'00"W          M
1       14110    Bahia Agua Fresca  53°23'00"S  70°58'00"W          V
2       14340    Bahia De Valdivia  39°48'00"S  73°15'00"W          S
3       14500  Bahia De Valparaiso  33°02'00"S  71°37'00"W          M
4       14140         Bahia Harris  53°51'00"S  70°27'00"W          V


In [28]:
import re
import pandas as pd

def dms_to_decimal(dms_str, lon_correction=0.008):

    if pd.isna(dms_str) or not isinstance(dms_str, str):
        return None

    try:
        # Limpiar símbolos y espacios
        s = (dms_str.strip()
                     .replace('"', '')
                     .replace("º", "°")
                     .replace("′", "'")
                     .replace(",", ".")
                     .replace("”", "")
                     .replace("’", "'"))
        s = re.sub(r'\s+', '', s)

        # Extraer grados, minutos, segundos y dirección
        match = re.match(r'(\d+(?:\.\d+)?)°(\d+(?:\.\d+)?)?\'?(\d+(?:\.\d+)?)?([NSEW])', s, re.IGNORECASE)
        if not match:
            return None

        deg, minutes, seconds, direction = match.groups()
        deg = float(deg)
        minutes = float(minutes) if minutes else 0.0
        seconds = float(seconds) if seconds else 0.0

        dec = deg + minutes / 60 + seconds / 3600

        # Aplicar signo según hemisferio
        if direction.upper() in ['S', 'W']:
            dec = -dec

        # Aplicar corrección si es longitud
        if direction.upper() in ['E', 'W']:
            dec += lon_correction

        return round(dec, 8)

    except Exception:
        return None


In [29]:
df["latitude"] = df["latitude"].apply(dms_to_decimal)
df["longitude"] = df["longitude"].apply(dms_to_decimal)

# Filtrar filas que no se pudieron convertir
df = df.dropna(subset=["latitude", "longitude"])

In [30]:
df.to_csv("Bases/puertos.csv", index=False)